In [5]:
import folium
from folium import GeoJson
import geopandas as gpd
from shapely.geometry import Polygon, mapping
import math


In [6]:
# Define the map's center location (latitude, longitude)
university_map = folium.Map(location=[6.7970, 79.9008], zoom_start=18)  # Example for University of Moratuwa

# Example: Adding new polygon coordinates (in degrees)
new_parking_slot_coords = [
    [79.899850, 6.797205],  # Top-left
    [79.899850, 6.797180],  # Bottom-left
    [79.900850, 6.797105],  # Bottom-right
    [79.900850, 6.797130]   # Top-right
]
new_parking_slot_coords1 = [
    [79.900956, 6.796610],  # Bottom-left
    [79.900944, 6.796620],  # Top-left
    [79.901223, 6.796878],  # Bottom-right
    [79.901233, 6.796868]   # Top-right
]
new_parking_slot_coords2 = [
    [79.900864, 6.796537],  # Bottom-left
    [79.900845, 6.796537],  # Top-left
    [79.900883, 6.796981],
    [79.900904, 6.796981],  # Bottom-right
       # Top-right
]
new_parking_slot_coords3 = [
    [79.899719, 6.7972070],  # Start from the bottom-left
    # [79.899895, 6.7972107],  # Move to the top-left
    [79.899840, 6.7972310],  # Slightly to the right
    [79.899943, 6.7972343],  # Continue up
    [79.900013, 6.7972343],  # Top-right corner
    [79.900025, 6.7973049],  # Down to the right-most point
    [79.899937, 6.7973149],  # Move leftwards
    [79.899928, 6.7972996],  # Slightly down
    [79.899910, 6.7972976],  # Continue left ***
    [79.899861, 6.7973618],  # Top-right of the cluster
    [79.899789, 6.7973452],  # Bottom-left corner of upper cluster
    [79.899758, 6.7973169],  # Downwards
    [79.899761, 6.7973026],  # Bottom-right of the middle cluster
    # [79.899778, 6.7972266],  # Return to the start cluster
    [79.899718, 6.7972500]  # Finally close the polygon
]

In [7]:
# Convert coordinates into a polygon using Shapely for CRS handling and accuracy
polygon = Polygon(new_parking_slot_coords)
polygon1 = Polygon(new_parking_slot_coords1)
polygon2 = Polygon(new_parking_slot_coords2)
polygon3 = Polygon(new_parking_slot_coords3)

In [8]:
# Create a GeoDataFrame with the polygon and ensure it is in EPSG:4326 (WGS84)
gdf = gpd.GeoDataFrame({'geometry': [polygon]})
gdf.set_crs("EPSG:4326", allow_override=True, inplace=True)

gdf1 = gpd.GeoDataFrame({'geometry': [polygon1]})
gdf1.set_crs("EPSG:4326", allow_override=True, inplace=True)

gdf2 = gpd.GeoDataFrame({'geometry': [polygon2]})
gdf2.set_crs("EPSG:4326", allow_override=True, inplace=True)

gdf3 = gpd.GeoDataFrame({'geometry': [polygon3]})
gdf3.set_crs("EPSG:4326", allow_override=True, inplace=True)

,geometry
0,"POLYGON ((79.89972 6.79721, 79.89984 6.79723, ..."


In [9]:

# Convert the GeoDataFrame into GeoJSON format for Folium
geojson = gdf.geometry.apply(mapping).iloc[0]
geojson1 = gdf1.geometry.apply(mapping).iloc[0]
geojson2 = gdf2.geometry.apply(mapping).iloc[0]
geojson3 = gdf3.geometry.apply(mapping).iloc[0]

In [10]:
# Create the GeoJson feature with properties
new_parking_slot = {
    "type": "Feature",
    "properties": {"name": "New Slot"},
    "geometry": geojson
}
new_parking_slot1 = {
    "type": "Feature",
    "properties": {"name": "New Slot1"},
    "geometry": geojson1
}
new_parking_slot2 = {
    "type": "Feature",
    "properties": {"name": "New Slot1"},
    "geometry": geojson2
}
new_parking_slot3 = {
    "type": "Feature",
    "properties": {"name": "New Slot1"},
    "geometry": geojson3
}

In [11]:
# Add the new polygon as a GeoJson layer to the map
GeoJson(new_parking_slot).add_to(university_map)
GeoJson(new_parking_slot1).add_to(university_map)
GeoJson(new_parking_slot2).add_to(university_map)
GeoJson(new_parking_slot3).add_to(university_map)

In [ ]:
# Display the map
university_map

In [22]:
# Define the polygon coordinates (already rearranged for a rectangle)
new_parking_slot_coords = [
    [79.899850, 6.797100],  # Top-left
    [79.899850, 6.797055],  # Bottom-left
    [79.900850, 6.797055],  # Bottom-right
    [79.900850, 6.797100]   # Top-right
]

In [23]:
# Create a Polygon object using the coordinates
polygon = Polygon(new_parking_slot_coords)

In [16]:
# Check if the polygon is valid
if polygon.is_valid:
    # Create a GeoDataFrame
    gdf = gpd.GeoDataFrame(geometry=[polygon])

    # Set the initial coordinate reference system (WGS84) for lat/lon
    gdf = gdf.set_crs("EPSG:4326")  # WGS84

    # Reproject to a projected coordinate system for accurate distance measurements
    # Using EPSG:3395 for Mercator projection (meters)
    gdf = gdf.to_crs(epsg=3395)  # Mercator projection (EPSG:3395)

    # Calculate area in square meters
    area = gdf.area[0]  # The first (and only) polygon
    print(f"Area of the polygon: {area:.2f} square meters")

    # Get the bounding box (minx, miny, maxx, maxy) in the projected CRS (meters)
    minx, miny, maxx, maxy = gdf.bounds.iloc[0]  # Use the GeoDataFrame's bounds

    # Calculate the width and length in meters using the bounds in the projected CRS
    width = maxx - minx  # Difference in x (East-West direction)
    length = maxy - miny  # Difference in y (North-South direction)

    print(f"Width of the polygon: {width:.2f} meters")
    print(f"Length of the polygon: {length:.2f} meters")
else:
    print("Invalid polygon!")

Area of the polygon: 557.88 square meters
Width of the polygon: 111.32 meters
Length of the polygon: 5.01 meters


In [17]:


def latlon_to_cartesian(center_lat, center_lon, points):
    """
    Convert geographic coordinates (latitude, longitude) to Cartesian (x, y) coordinates in meters.

    Parameters:
        center_lat (float): Latitude of the Cartesian plane's origin (0, 0).
        center_lon (float): Longitude of the Cartesian plane's origin (0, 0).
        points (list): List of geographic coordinates as [(lon, lat), ...].

    Returns:
        list: List of Cartesian coordinates as [(x, y), ...].
    """
    # Earth's radius in meters
    R = 6371000
    center_lat_rad = math.radians(center_lat)
    center_lon_rad = math.radians(center_lon)

    cartesian_coords = []
    for lon, lat in points:
        # Convert to radians
        lat_rad = math.radians(lat)
        lon_rad = math.radians(lon)

        # Calculate x and y using the Haversine formula
        delta_lat = lat_rad - center_lat_rad
        delta_lon = lon_rad - center_lon_rad
        x = R * delta_lon * math.cos(center_lat_rad)
        y = R * delta_lat
        cartesian_coords.append((x, y))

    return cartesian_coords



In [18]:

# Example usage
# Center of the Cartesian plane (University of Moratuwa)
center_lat = 6.7971657  # Latitude
center_lon = 79.899895  # Longitude


In [19]:

# Polygon vertices in (longitude, latitude)
new_parking_slot_coords = [
    [79.899850, 6.797205],  # Top-left
    [79.899850, 6.797180],  # Bottom-left
    [79.900850, 6.797105],  # Bottom-right
    [79.900850, 6.797130]   # Top-right
]


In [20]:

# Convert to Cartesian (x, y)
cartesian_polygon = latlon_to_cartesian(center_lat, center_lon, new_parking_slot_coords)


In [21]:

# Output the Cartesian coordinates
print("Cartesian Polygon Coordinates (in meters):")
print(cartesian_polygon)

Cartesian Polygon Coordinates (in meters):
[(-4.968602009502883, 4.3699606171159475), (-4.968602009502883, 1.5900874509738594), (105.44477598101084, -6.74953204736399), (105.44477598101084, -3.969658881221902)]
